# Bottom up method; From power plant data to country aggregated CO2 intensity of electricity generation

In this script we calculated an CO2 emission factor per country out of hourly power plant generation and yearly published emission data on plant level.

The used method follows the idea to calculated a CI for specific power plants. In a second step an representative sample of power plants for a country is build and an CI for each technology and country is calculated.

# Script setup

In [126]:
import os
import logging

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from IPython.display import Image 

%matplotlib inline
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = [15, 10]

#helpers


# Data directory preparention

In [2]:
# Create input, processed and output folders if they don't exist
# If the paths are relative, the correspoding folders will be created inside the current working directory.

input_directory_path = os.path.join('input')
Bootom_up_methode_input_directory_path = os.path.join('input', 'Bootom_up_methode')
processed_directory_path = 'processed'
output_directory_path = os.path.join('output')

os.makedirs(input_directory_path, exist_ok=True)
os.makedirs(Bootom_up_methode_input_directory_path, exist_ok=True)
os.makedirs(processed_directory_path, exist_ok=True)
os.makedirs(output_directory_path, exist_ok=True)

# Data file preperation

The directory `input/Bootom_up_method` should contain all necessary raw data files.

Based on the developed method the following data sets needed:

1) Matcher for power plants in Entso e and EUTL Data for Germany
Matching List for German power plants with Entso e identifier and the EUTL identifier.
Data download form: https://zenodo.org/record/3588418#.XxlZOufgq5h \

File - > Matching_Entso_EUTL_LCPD.csv

corresponding Paper: "Comparing empirical and model-based approaches for calculating dynamic grid emission factors: an application to CO2-minimizing storage dispatch in Germany"
https://linkinghub.elsevier.com/retrieve/pii/S0959652620316358

2) EU Emissions Data (EUTL)
Data are provided in the report section. See the following link:
https://ec.europa.eu/clima/policies/ets/registry_en#tab-0-1

For example: ->Documentation->Reports->Verified Emissions for 2019

File - > "Verified Emissions for 2019" -> verified_emissions_2018_en.xlsx -> converted to .csv

Another way to check the data is directly through the European Union Transaction Log:
https://ec.europa.eu/clima/ets/napMgt.do?languageCode=en

3) ENTSO e Data
Production per Unit from ENTSO-E
Original data: ENTSO-E Transparency Platform, Actual Generation per Generation Unit Available online: https://transparency.entsoe.eu/generation/r2/actualGenerationPerGenerationUnit/show (accessed on Apr 29, 2020).
Processed with the following script that converts the data to hourly resolution (private script from INATECH):
https://github.com/INATECH-CIG/entso-e_GenerationOutputPerUnit


File - > gen_data.csv -> hourly generation data per unit\

File - > unit_data.csv -> information about the generation units

4) Entso unit generation data from EMBER
The Entso e data has lot of gaps and need to be preprocessing (is done partly and the result is different than the one from Ember)

File - > entsoe_unit_gen_data.csv

5) Matcher for power plants in Entso e and EUTL Data for Europe
The matching was done by one of our students

File - > Matching_Entso_EUTL_EU.csv


# Load data functions

In [43]:
def load_matching_data(path, fn):
    """
    Matching List for German power plants with Entso e identifier and the EUTL identifier.
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn), sep = ',', header = 0, encoding = 'unicode_escape')

    return df

def load_EUTL_data(path, fn):
    """
    EU Emissions Data (EUTL)
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn),sep = ';',header = 13,encoding ='unicode_escape' )

    return df

def load_generation_data(path, fn):
    """
    Entso e gernation data per unit
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=0,parse_dates=True)

    return df

def load_unit_data(path, fn):
    """
    Entso e generation unit information
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn),sep = ',',index_col=0)
    # set name for the index
    df.index.set_names('GenerationUnitEIC', inplace=True)

    return df

def load_unit_data_ember(path, fn):
    """
    Entso e unit generation from ember
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn), sep = ';', header = 0, encoding = 'unicode_escape')

    return df

def load_matching_data_EU(path, fn):
    """
    Matching List for EU power plants with Entso e identifier and the EUTL identifier.
        
    Parameters
    ----------
    path: str
        path to data
    fn : str
        filename
        
    """
    
    logging.info(f'Loading data from {fn}')
    
    df = pd.read_csv(os.path.join(path, fn), sep = ',', header = 0, index_col=0)

    return df

# load data sets

In [4]:
generation_per_unit = load_generation_data(Bootom_up_methode_input_directory_path, 'gen_data.csv')

In [5]:
generation_unit_info = load_unit_data(Bootom_up_methode_input_directory_path, 'unit_data.csv')

In [6]:
EUTL_emissions = load_EUTL_data(Bootom_up_methode_input_directory_path, 'verified_emissions_2018_en.csv')

In [7]:
unit_matching_DE = load_matching_data(Bootom_up_methode_input_directory_path, 'Matching_Entso_EUTL_LCPD.csv')

In [24]:
generation_per_unit_ember = load_unit_data_ember(Bootom_up_methode_input_directory_path, 'entsoe_unit_gen_data.csv') 

#generation_per_unit_ember = generation_per_unit_ember[generation_per_unit_ember.Year == 2018]

#generation_per_unit_ember = generation_per_unit_ember.groupby(by = 'PowerSystemResourceName').sum()['Monthly Generation (GWh)']

In [116]:
unit_matching_EU = load_matching_data_EU(Bootom_up_methode_input_directory_path, 'Matching_Entso_EUTL_EU.csv')

In [117]:
unit_matching_EU_2 = load_matching_data_EU(Bootom_up_methode_input_directory_path, 'Matching_Entso_EUTL_EU_2.csv')

# first setps

calculation of yearly generation for units

In [8]:
generation_unit_info['yearly_generation_2018'] = generation_per_unit.sum()

vergleich unsere Jährliche Erzeugung mit denen, die wir von Ember bekommen haben (in 12, in Gfuellner Code) . Die unserscheidet sich. Was macht Ember noch mit den Daten?

In [9]:
generation_unit_info[generation_unit_info.PowerSystemResourceName == 'Scheldelaan Exxonmobil']

,AreaCode,AreaName,AreaTypeCode,InstalledGenCapacity,MapCode,PowerSystemResourceName,ProductionTypeName,duplicate_count,yearly_generation_2018
GenerationUnitEIC,,,,,,,,,
22WSCHELD1502261,10YBE----------2,Elia CA,CTA,140.0,BE,Scheldelaan Exxonmobil,Fossil Gas,1.0,809517.51


Match EU Emission Code to power plants

In [120]:
merged_EU = pd.merge(generation_unit_info, unit_matching_EU, on='PowerSystemResourceName', how='inner')

In [121]:
merged_EU

,AreaCode,AreaName,AreaTypeCode,InstalledGenCapacity,MapCode,PowerSystemResourceName,ProductionTypeName,duplicate_count,yearly_generation_2018,countrycode,EUTL_ID
0,10YAT-APG------L,APG CA,CTA,140.0,AT,Lau GuD,Fossil Gas,2.0,0.000,AT,86.0
1,10YAT-APG------L,APG CA,CTA,400.0,AT,Kraftwerk Timelkam GUD,Fossil Gas,2.0,685235.670,AT,149.0
2,10YAT-APG------L,APG CA,CTA,332.0,AT,KW Dürnrohr Block 2,Fossil Hard coal,2.0,745290.410,AT,94.0
3,10YAT-APG------L,APG CA,CTA,150.0,AT,KW Riedersbach 2 G2,Fossil Hard coal,2.0,0.000,AT,79.0
4,10Y1001A1001A796,Energinet CA,CTA,147.0,DK,Asnaesvaerket 2,Fossil Hard coal,1.0,219309.910,DK,48.0
...,...,...,...,...,...,...,...,...,...,...,...
796,10YGB----------A,National Grid CA,CTA,525.0,GB,EGGPS-3,Fossil Hard coal,0.0,236678.180,GB,169.0
797,10YGB----------A,National Grid CA,CTA,260.0,GB,DEEP-1,Fossil Gas,0.0,335757.515,GB,187.0
798,10YGB----------A,National Grid CA,CTA,525.0,GB,EGGPS-2,Fossil Hard coal,0.0,15438.130,GB,169.0
799,10YGB----------A,National Grid CA,CTA,20.0,GB,WBUGT-4,Fossil Gas,0.0,81.230,GB,145.0


In [82]:
generation_per_unit_ember = generation_per_unit_ember[generation_per_unit_ember.Year == 2018]

,Date,Year,Month,MapCode,PowerSystemResourceName,ProductionTypeName,Installed Capacity (MW),Average Generation (MW),Load Factor (%),Monthly Generation (GWh),Missing hours (%)
41510,01.01.2018,2018,1,AL,FIERZAG1,Hydro Water Reservoir,NaN,NaN,NaN,NaN,100
41511,01.01.2018,2018,1,AL,FIERZAG2,Hydro Water Reservoir,NaN,NaN,NaN,NaN,100
41512,01.01.2018,2018,1,AL,FIERZAG3,Hydro Water Reservoir,NaN,NaN,NaN,NaN,100
41513,01.01.2018,2018,1,AL,FIERZAG4,Hydro Water Reservoir,NaN,NaN,NaN,NaN,100
41514,01.01.2018,2018,1,AL,KOMANG1,Hydro Water Reservoir,NaN,NaN,NaN,NaN,100
...,...,...,...,...,...,...,...,...,...,...,...
62569,01.12.2018,2018,12,SK,Nováky TG2,Fossil Brown coal/Lignite,110.0,84.0,76,62.0,0
62570,01.12.2018,2018,12,SK,Nováky TG3,Fossil Brown coal/Lignite,110.0,0.0,0,0.0,0
62571,01.12.2018,2018,12,SK,PPC Bratislava TG1,Fossil Gas,158.0,0.0,0,0.0,0
62572,01.12.2018,2018,12,SK,Vojany TG5,Fossil Hard coal,110.0,44.0,40,33.0,0


In [84]:
generation_per_unit_ember = generation_per_unit_ember.groupby(by = 'PowerSystemResourceName').agg({'Monthly Generation (GWh)': 'sum',
                                                                       'MapCode':'first',
                                                                       'ProductionTypeName':'first',
                                                                       
                                                                         'Year': 'first',
                                                                         'Installed Capacity (MW)': 'first'
                                                                      })

In [124]:
merged_ember = pd.merge(generation_per_unit_ember, unit_matching_EU_2, on='PowerSystemResourceName', how='inner')

In [125]:
merged_ember

,PowerSystemResourceName,Monthly Generation (GWh),MapCode,ProductionTypeName,Year,Installed Capacity (MW),countrycode,EUTL_ID
0,ABOÑO 1,1667.0,ES,Fossil Hard coal,2018,342.0,ES,201.0
1,ABOÑO 2,3311.0,ES,Fossil Hard coal,2018,536.0,ES,201.0
2,ABTH7,0.0,GB,Fossil Hard coal,2018,535.0,GB,188.0
3,ABTH8,0.0,GB,Fossil Hard coal,2018,535.0,GB,188.0
4,ABTH9,0.0,GB,Fossil Hard coal,2018,520.0,GB,188.0
...,...,...,...,...,...,...,...,...
848,Wilhelmshaven,1761.0,DE_TenneT_GER,Fossil Hard coal,2018,757.0,DE,969.0
849,Wroclaw Bl2,348.0,PL,Fossil Hard coal,2018,108.0,PL,91.0
850,Wroclaw Bl3,437.0,PL,Fossil Hard coal,2018,108.0,PL,91.0
851,Zandvliet Power,2031.0,BE,Fossil Gas,2018,386.0,BE,307.0


In [96]:
EUTL_emissions

,REGISTRY_CODE,IDENTIFIER_IN_REG,INSTALLATION_NAME,INSTALLATION_IDENTIFIER,PERMIT_IDENTIFIER,MAIN_ACTIVITY_TYPE_CODE,ALLOCATION_2018,ALLOCATION_RESERVE_2018,ALLOCATION_TRANSITIONAL_2018,VERIFIED_EMISSIONS_2018,...,VERIFIED_EMISSIONS_2012,ALLOCATION_2011,VERIFIED_EMISSIONS_2011,ALLOCATION_2010,VERIFIED_EMISSIONS_2010,ALLOCATION_2009,VERIFIED_EMISSIONS_2009,ALLOCATION_2008,VERIFIED_EMISSIONS_2008,ACCOUNT_CLOSURE
0,AT,Michael Strasser GmbH & Co. KG,13030,201505.0,BMLFUW?UW.1.3.2/0121?V/4/2011,10,-1,-1,-1,-1,...,21,-1,-1,-1,-1,-1,-1,-1,-1,29. Aug 14
1,AT,Georgetown Management LLC,15451,201836.0,BMLFUW-UW.1.3.2/0121-V/4/2010,10,-1,-1,-1,Excluded,...,31,-1,-1,-1,-1,-1,-1,-1,-1,OPEN
2,AT,PSC Ukraine International Airlines,19210,201564.0,BMLFUW-UW.1.3.2/0001-I/4/2015,10,3286,0,0,67,...,6165,-1,-1,-1,-1,-1,-1,-1,-1,OPEN
3,AT,Glock Gesellschaft m.b.H.,194,200108.0,BMFLUW-UW.1.3.2/0084-V/4/2010,10,-1,-1,-1,-1,...,1678,-1,-1,-1,-1,-1,-1,-1,-1,23. Dez 15
4,AT,Air Charter Limited,209,200180.0,BMFLUW-UW.1.3.2/0372-V/4/2009,10,-1,-1,-1,Excluded,...,297,-1,-1,-1,-1,-1,-1,-1,-1,OPEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14109,SK,Elektrárne Vojany,Závod Elektrárne Vojany,109.0,807-004-2017,20,1763,0,0,710588,...,600273,2901200,573432,2901200,645958,2901200,898417,2901200,1583830,OPEN
14110,SK,Wienerberger - závod Zlaté Moravce,Závod Zlaté Moravce,159.0,407-001-2012,32,11539,0,0,14358,...,9641,15215,11746,15215,8302,15215,9735,15215,12849,OPEN
14111,SK,"MESTSKÝ BYTOVÝ PODNIK, s.r.o.",kotol?a Sedlite,183.0,308-002-2012,20,4421,0,0,8537,...,9821,12038,9780,12038,10916,12038,10502,12038,10533,OPEN
14112,SK,IAROMAT a.s.,IAROMAT a.s.,190.0,607-009-2017,32,3290,0,0,3307,...,3110,4694,3162,4694,3078,4694,2789,4694,3270,OPEN


In [99]:
def conv(x):
    # converts german grid operator areas string in german countrycode of ETS
    areas =['DE_TenneT_GER','DE_TransnetBW','DE_Amprion','DE_50HzT']
    if x in areas:
        x = 'DE' 
    return x

def connect_CO2(df, ETS):
    #df = df.drop(columns = 'index')
    df['countrycode']= df.countrycode.apply(lambda x: conv(x))
    # caring for characteristics of the dataset: EUTL-ID IS NOT UNIQUE - only countrywise.
    # removing power plants from the dataset which are not covered in ETS:
    miss_country = []
    for i in set(df.countrycode):
        if sum(i == ETS['REGISTRY_CODE']) == 0:
            print(i)
            miss_country.append(i)
            #removing this power plants from the list:
    df = df.query('countrycode not in @miss_country')
    
    # removing power plants where match could not be found:
    miss_match = []
    for j in df.PowerSystemResourceName:
        row = df[df.PowerSystemResourceName == j]
        if len(ETS.query('REGISTRY_CODE == @row.countrycode.iloc[0]')\
                  .query('INSTALLATION_IDENTIFIER == @row.EUTL_ID.iloc[0]')) == 0:
            print(j)
            miss_match.append(j)
            
    df = df.query('PowerSystemResourceName not in @miss_match')
    # apply matching:
    df['verified_emissions_18'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['VERIFIED_EMISSIONS_2018']\
                                    .iloc[0], axis = 1)
    df['verified_emissions_17'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['VERIFIED_EMISSIONS_2017']\
                                    .iloc[0], axis = 1)
    df['verified_emissions_16'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['VERIFIED_EMISSIONS_2016']\
                                    .iloc[0], axis = 1)
    df['verified_emissions_15'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['VERIFIED_EMISSIONS_2015']\
                                    .iloc[0], axis = 1)
    
    df['ETS_name'] = df.apply(lambda x: ETS.query('REGISTRY_CODE==@x.countrycode')\
                                    .query('INSTALLATION_IDENTIFIER==@x.EUTL_ID')['IDENTIFIER_IN_REG']\
                                    .iloc[0], axis = 1)
    return df

In [100]:
Powerplants_emission = connect_CO2(merged_ember, EUTL_emissions)

In [102]:
Powerplants_emission_EU = connect_CO2(merged_EU, EUTL_emissions)

In [111]:
Powerplants_emission_EU

,AreaCode,AreaName,AreaTypeCode,InstalledGenCapacity,MapCode,PowerSystemResourceName,ProductionTypeName,duplicate_count,yearly_generation_2018,countrycode,EUTL_ID,verified_emissions_18,verified_emissions_17,verified_emissions_16,verified_emissions_15,ETS_name
0,10YAT-APG------L,APG CA,CTA,140.0,AT,Lau GuD,Fossil Gas,2.0,0.000,AT,86.0,6300.0,3825,3136,4097,Wienstrom KW Leopoldau Wien
1,10YAT-APG------L,APG CA,CTA,400.0,AT,Kraftwerk Timelkam GUD,Fossil Gas,2.0,685235.670,AT,149.0,267204.0,356994,213716,174692,Energie AG GuD Kraftwerk Timelkam
2,10YAT-APG------L,APG CA,CTA,332.0,AT,KW Dürnrohr Block 2,Fossil Hard coal,2.0,745290.410,AT,94.0,0.0,0,8,565110,Verbund KW Dürnrohr Zwentendorf
3,10YAT-APG------L,APG CA,CTA,150.0,AT,KW Riedersbach 2 G2,Fossil Hard coal,2.0,0.000,AT,79.0,6576.0,7775,203045,270681,KW Riedersbach
4,10Y1001A1001A796,Energinet CA,CTA,147.0,DK,Asnaesvaerket 2,Fossil Hard coal,1.0,219309.910,DK,48.0,878395.0,1048994,952588,695549,Asnæsværket
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,10YGB----------A,National Grid CA,CTA,525.0,GB,EGGPS-3,Fossil Hard coal,0.0,236678.180,GB,169.0,540497.0,1014334,2098992,4748504,Eggborough Operator Account
797,10YGB----------A,National Grid CA,CTA,260.0,GB,DEEP-1,Fossil Gas,0.0,335757.515,GB,187.0,145298.0,289810,248853,141965,Operator Account
798,10YGB----------A,National Grid CA,CTA,525.0,GB,EGGPS-2,Fossil Hard coal,0.0,15438.130,GB,169.0,540497.0,1014334,2098992,4748504,Eggborough Operator Account
799,10YGB----------A,National Grid CA,CTA,20.0,GB,WBUGT-4,Fossil Gas,0.0,81.230,GB,145.0,1679509.0,1766100,1165058,7724207,West Burton A


In [112]:
def calc_CI(df):
    # calculates the carbon intensity of sites. Due to non-unique labelings of the power plants across different
    # countries it needs care taken about it.
    
    df['verified_emissions_18']=df['verified_emissions_18'].apply(float)
    
    sites = pd.DataFrame(df.groupby(['countrycode','EUTL_ID']).mean()['verified_emissions_18']\
                       / df.groupby(['countrycode','EUTL_ID']).sum()['yearly_generation_2018'])\
                        .reset_index()
    df['carbon_intensity'] = df.apply(lambda x: sites.query('countrycode == @x.countrycode')\
                            .query('EUTL_ID == @x.EUTL_ID')[0].iloc[0],axis = 1)
    
    return df

In [113]:
Powerplants_emission_EU_CI = calc_CI(Powerplants_emission_EU)

In [114]:
Powerplants_emission_EU_CI

,AreaCode,AreaName,AreaTypeCode,InstalledGenCapacity,MapCode,PowerSystemResourceName,ProductionTypeName,duplicate_count,yearly_generation_2018,countrycode,EUTL_ID,verified_emissions_18,verified_emissions_17,verified_emissions_16,verified_emissions_15,ETS_name,carbon_intensity
0,10YAT-APG------L,APG CA,CTA,140.0,AT,Lau GuD,Fossil Gas,2.0,0.000,AT,86.0,6300.0,3825,3136,4097,Wienstrom KW Leopoldau Wien,inf
1,10YAT-APG------L,APG CA,CTA,400.0,AT,Kraftwerk Timelkam GUD,Fossil Gas,2.0,685235.670,AT,149.0,267204.0,356994,213716,174692,Energie AG GuD Kraftwerk Timelkam,0.389945
2,10YAT-APG------L,APG CA,CTA,332.0,AT,KW Dürnrohr Block 2,Fossil Hard coal,2.0,745290.410,AT,94.0,0.0,0,8,565110,Verbund KW Dürnrohr Zwentendorf,0.000000
3,10YAT-APG------L,APG CA,CTA,150.0,AT,KW Riedersbach 2 G2,Fossil Hard coal,2.0,0.000,AT,79.0,6576.0,7775,203045,270681,KW Riedersbach,inf
4,10Y1001A1001A796,Energinet CA,CTA,147.0,DK,Asnaesvaerket 2,Fossil Hard coal,1.0,219309.910,DK,48.0,878395.0,1048994,952588,695549,Asnæsværket,1.140746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,10YGB----------A,National Grid CA,CTA,525.0,GB,EGGPS-3,Fossil Hard coal,0.0,236678.180,GB,169.0,540497.0,1014334,2098992,4748504,Eggborough Operator Account,1.095420
797,10YGB----------A,National Grid CA,CTA,260.0,GB,DEEP-1,Fossil Gas,0.0,335757.515,GB,187.0,145298.0,289810,248853,141965,Operator Account,0.432747
798,10YGB----------A,National Grid CA,CTA,525.0,GB,EGGPS-2,Fossil Hard coal,0.0,15438.130,GB,169.0,540497.0,1014334,2098992,4748504,Eggborough Operator Account,1.095420
799,10YGB----------A,National Grid CA,CTA,20.0,GB,WBUGT-4,Fossil Gas,0.0,81.230,GB,145.0,1679509.0,1766100,1165058,7724207,West Burton A,9890.810047
